In [21]:

import psycopg2

def get_connection():
    return psycopg2.connect(
        dbname="platform",
        user="postgres",
        password="password",
        host="127.0.0.1",
        port="5432"
    )

In [22]:
connection = get_connection()
with connection.cursor() as cursor:
    cursor.execute(
        """
        CREATE TABLE IF NOT EXISTS bank (id SERIAL PRIMARY KEY, account VARCHAR(255), balance FLOAT);
        """
    )

connection.commit()
connection.close()

# ACID 

* A - atomicity
* C - Consistency 
* I - Isolation
* D - Durability 

## Atomicity 
Either all the operations within the transaction are executed successfully, or none of them are. There is no in-between state where some operations have completed and others have not.

* All-or-Nothing Principle:
* Rollback Mechanism:
* Commit:
* Failure Handling:

In [23]:
connection = get_connection()

with connection.cursor() as cursor:
    cursor.execute("INSERT INTO bank (account, balance) VALUES ('Alice', 100);")

connection.commit()
connection.close()

In [26]:
connection = get_connection()

with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM bank;")
    for row in cursor.fetchall():
        print(row)

connection.close()

(1, 'Alice', 100.0)


In [25]:
connection = get_connection()

try:
    with connection.cursor() as cursor:
        cursor.execute("INSERT INTO bank (account, balance) VALUES ('Dan', 100);")
        error = 1/0
    connection.commit()
finally:
    connection.close()

ZeroDivisionError: division by zero

## Consistency
It is forcing the rules.\
It ensures that all database transactions adhere to the predefined rules and constraints.

In [27]:
connection = get_connection()
with connection.cursor() as cursor:
    cursor.execute(
        """
        ALTER TABLE bank ADD CONSTRAINT unique_account UNIQUE (account);
        """
    )
connection.commit()
connection.close()

In [28]:
connection = get_connection()

with connection.cursor() as cursor:
    cursor.execute("INSERT INTO bank (account, balance) VALUES ('Alice', 100);")

connection.commit()
connection.close()

UniqueViolation: duplicate key value violates unique constraint "unique_account"
DETAIL:  Key (account)=(Alice) already exists.


## Isolation
Isolation in ACID ensures that transactions do not interfere with each other, maintaining a state as if transactions were executed sequentially.

In [29]:
connection1 = get_connection()

In [35]:
connection1.commit()

In [37]:
connection1.close()

In [33]:
with connection1.cursor() as cursor:
    cursor.execute("SELECT * FROM bank;")
    for row in cursor.fetchall():
        print(row)

(1, 'Alice', 100.0)
(4, 'Bob', 100.0)


In [36]:
connection = get_connection()

with connection.cursor() as cursor:
    cursor.execute("SELECT * FROM bank;")
    for row in cursor.fetchall():
        print(row)

connection.close()

(1, 'Alice', 100.0)
(4, 'Bob', 100.0)


In [32]:
with connection1.cursor() as cursor:
    cursor.execute("INSERT INTO bank (account, balance) VALUES ('Bob', 100);")

## Durability
Once a transaction is committed, the changes made to the database will persist, regardless of power outages, crashes, or other failures. 